In [2]:
import os
import numpy as np
import re
import keras
import tensorflow as tf
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


A function from set 6 to parse the observations:

In [3]:
def parse_observations(text):
    # Convert text to dataset.
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            word = re.sub(r'[^\w]', '', word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

In [4]:
text = open(os.path.join(os.getcwd(), \
                         'Release/data/shakespeare.txt')).read()
obs, obs_map = parse_observations(text)

Since we’re doing a character-based LSTM, first, we create a sorted dictionary of all characters used in the entire text, where each character is assigned an index. We create a two dictionaries, a character-to-index dictionary and an index-to-character dictionary, which will be used later for easy access to characters and indices.

In [5]:
all_chars = list(text)
chars = sorted(list(set(text)))
char_to_index = {char:idx for idx, char in enumerate(chars)}
index_to_char = {idx:char for idx, char in enumerate(chars)}

We break up the text into 40-character-length sequences, which will be our training X’s. For each 40 character sequence, the corresponding training Y will be the next character that follows. These input pairs will be the data that we train the LSTM on. In our code, we can also specify a step size (n_step) so that we can take the 40-character sequences at different intervals. 

In [5]:
n_step = 2
s_len = 40
training_data = []
next_char = []
for i in range(0, len(text)-s_len, n_step):
    string_to_add = text[i:i + s_len]
    training_data.append(string_to_add)
    if i + s_len < len(text)-1:
        next_char.append(text[i + s_len])
    
# This is our end symbol: a space
next_char.append(' ')

The Keras LSTM takes a 3-dimensional input for X and a 2-dimensional input for Y. The dimensions of the X input are [number of sequences x sequence length(40) x length of character vector]. The dimensions of Y are [number of sequences x length of character vector]. The X-input is the list of 40-character sequences, where each position contains the vectorized character. The Y-input is the list of characters following the 40-char sequences, where each character is also vectorized. We manually 1-hot vectorize the characters. 

In [6]:
trainX = np.zeros((len(training_data), s_len, len(chars)))
trainY = np.zeros((len(training_data), len(chars)))

for sent_idx in range(0, len(training_data)):
    curr_sentence = training_data[sent_idx]
    curr_next_char = next_char[sent_idx]
    for i, char in enumerate(curr_sentence):
        trainX[sent_idx, i, char_to_index[char]] = 1
    trainY[sent_idx, char_to_index[curr_next_char]] = 1

Define functions for printing out poems from this LSTM.

In [7]:
def print_poem_maxprob(length, random = False):
    if random == True:
        start_index = np.random.randint(0, len(text)-s_len-1)
        start_index = 50
        sentence = text[start_index: start_index + s_len]
    else:
        sentence = "shall i compare thee to a summer's day?\n"
    sequence = sentence
    for i in range(length):
        x_pred = np.zeros((1, s_len, len(chars)))
        for j, char in enumerate(sentence):
            x_pred[0, j, char_to_index[char]] = 1.

        predictions = np.array(model.predict(x_pred)[0])
        max_index = np.argmax(predictions)
        next_char = index_to_char[max_index]
        sentence = sentence[1:] + next_char
        sequence += next_char

    print(sequence)

In [8]:
def print_poem_randprob(length, random = False):
    if random == True:
        start_index = np.random.randint(0, len(text)-s_len-1)
        start_index = 50
        sentence = text[start_index: start_index + s_len]
    else:
        sentence = "shall i compare thee to a summer's day?\n"
    sequence = sentence
    for i in range(length):
        x_pred = np.zeros((1, s_len, len(chars)))
        for j, char in enumerate(sentence):
            x_pred[0, j, char_to_index[char]] = 1.

        predictions = np.array(model.predict(x_pred)[0])
        index = np.random.choice(len(chars), 1, p=predictions)[0]
        next_char = index_to_char[index]
        sentence = sentence[1:] + next_char
        sequence += next_char

    print(sequence)

In this model, we use a bidirectional LSTM layers of 120 units, with a dropout of 0.2. It has a standard fully-connected output layer with a softmax activation. We train it for 40 epochs. We print out the poems generated by selecting the maximum probability letter and also the poems generated by selecting a random character weighted based on the probability.

In [13]:
model = Sequential()
model.add(Bidirectional(LSTM(120),
                        input_shape=(s_len, len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

#train the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop')
model.fit(trainX, trainY, batch_size = 128, nb_epoch = 40)
print_poem_maxprob(1000)
print_poem_randprob(1000)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/40
48995/48995 [==============================] - 53s 1ms/step - loss: 2.9191
Epoch 2/40
48995/48995 [==============================] - 46s 943us/step - loss: 2.4056
Epoch 3/40
48995/48995 [==============================] - 43s 885us/step - loss: 2.2049
Epoch 4/40
48995/48995 [==============================] - 44s 904us/step - loss: 2.1050
Epoch 5/40
48995/48995 [==============================] - 43s 885us/step - loss: 2.0367
Epoch 6/40
48995/48995 [==============================] - 43s 884us/step - loss: 1.9781
Epoch 7/40
48995/48995 [==============================] - 43s 885us/step - loss: 1.9314
Epoch 8/40
48995/48995 [==============================] - 43s 886us/step - loss: 1.8908
Epoch 9/40
48995/48995 [==============================] - 43s 883us/step - loss: 1.8577
Epoch 10/40
48995/48995 [==============================] - 44s 892us/step - loss: 1.8254
Epoch 11/40
48995/48995 [==============================] - 44s 889us/step - loss: 1.7969
Epoch 12/40
48995/48995 [=======

In this model, we train a simple RNN layer with 100 units, with a dropout of 0.2. It has a standard fully-connected output layer with a softmax activation. We train it for 50 epochs. We print out the poems generated by selecting the maximum probability letter and also the poems generated by selecting a random character weighted based on the probability.

In [20]:
model = Sequential()
model.add(SimpleRNN(100, input_shape=(s_len, len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

#train the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop')
model.fit(trainX, trainY, batch_size = 128, nb_epoch = 50)
print_poem_maxprob(1000)
print_poem_randprob(1000)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
48995/48995 [==============================] - 9s 191us/step - loss: 2.8478
Epoch 2/50
48995/48995 [==============================] - 8s 170us/step - loss: 2.3959
Epoch 3/50
48995/48995 [==============================] - 8s 171us/step - loss: 2.2499
Epoch 4/50
48995/48995 [==============================] - 8s 171us/step - loss: 2.1795
Epoch 5/50
48995/48995 [==============================] - 8s 167us/step - loss: 2.1339
Epoch 6/50
48995/48995 [==============================] - 8s 166us/step - loss: 2.1048
Epoch 7/50
48995/48995 [==============================] - 8s 167us/step - loss: 2.0828
Epoch 8/50
48995/48995 [==============================] - 8s 169us/step - loss: 2.0655
Epoch 9/50
48995/48995 [==============================] - 8s 166us/step - loss: 2.0516
Epoch 10/50
48995/48995 [==============================] - 8s 167us/step - loss: 2.0388
Epoch 11/50
48995/48995 [==============================] - 8s 167us/step - loss: 2.0274
Epoch 12/50
48995/48995 [================

In this model, we use a GRU recurrent network layer of 120 units, with a dropout of 0.2. It has a standard fully-connected output layer with a softmax activation. We train it for 50 epochs. We print out the poems generated by selecting the maximum probability letter and also the poems generated by selecting a random character weighted based on the probability.

In [21]:
model = Sequential()
model.add(GRU(120, input_shape=(s_len, len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

#train the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam')
model.fit(trainX, trainY, batch_size = 128, nb_epoch = 50)
print_poem_maxprob(1000)
print_poem_randprob(1000)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
48995/48995 [==============================] - 26s 533us/step - loss: 2.8573
Epoch 2/50
48995/48995 [==============================] - 23s 470us/step - loss: 2.2418
Epoch 3/50
48995/48995 [==============================] - 23s 470us/step - loss: 2.0789
Epoch 4/50
48995/48995 [==============================] - 23s 469us/step - loss: 1.9884
Epoch 5/50
48995/48995 [==============================] - 23s 471us/step - loss: 1.9224
Epoch 6/50
48995/48995 [==============================] - 23s 476us/step - loss: 1.8718
Epoch 7/50
48995/48995 [==============================] - 23s 470us/step - loss: 1.8282
Epoch 8/50
48995/48995 [==============================] - 23s 472us/step - loss: 1.7871
Epoch 9/50
48995/48995 [==============================] - 23s 475us/step - loss: 1.7538
Epoch 10/50
48995/48995 [==============================] - 23s 476us/step - loss: 1.7236
Epoch 11/50
48995/48995 [==============================] - 23s 476us/step - loss: 1.6965
Epoch 12/50
48995/48995 [=====

Analysis with variation of temperature.

In [55]:
def print_poem_temp(length, temp, random = False):
    if random == True:
        start_index = np.random.randint(0, len(text)-s_len-1)
        start_index = 50
        sentence = text[start_index: start_index + s_len]
    else:
        sentence = "shall i compare thee to a summer's day?\n"
    sequence = sentence
    for i in range(length):
        x_pred = np.zeros((1, s_len, len(chars)))
        for j, char in enumerate(sentence):
            x_pred[0, j, char_to_index[char]] = 1.

        predictions = np.array(model.predict(x_pred)[0])
        for p in predictions:
            p = np.exp(p/temp)
        div = sum(predictions)
        for p in predictions:
            p = p/np.sum(predictions)
        #predictions = preds
        #max_index = np.argmax(predictions)
        index = np.random.choice(len(chars), 1, p=predictions)[0]
        next_char = index_to_char[index]
        sentence = sentence[1:] + next_char
        sequence += next_char

    print(sequence)


In [57]:
print("Testing Temperature of 1.5....")
print_poem_temp(1000, 1.5, random = False)
print("\n")

print("Testing Temperature of 0.75....")
print_poem_temp(1000, 0.75, random = False)
print("\n")

print("Testing Temperature of 0.25....")
print_poem_temp(1000, 0.25, random = False)
print("\n")


Testing Temperature of 1.5....
shall i compare thee to a summer's day?
So surmind one all more for praises will,
Or and all here with it love, duspitit sweet:
O 8lone thou forsique owmy thou from suy,
Sole becuring hap yell days surot minit,
Trey preight one, and chould how frows wall,
When So thee over foul sispy thy doot hide,
And to stymy gave what shish I man.
  Unlow mire int not desarget that I foul,
So thou gively my failtr theavince ching,
Will not thou veity notber's lived earth,
Whal her apceail,'ncoure her five onder sprense
This the beak you rotere so thured so trife,
  And sefelt thou pidque olme hane I mure,
Nor forgater's band or fils in my bruth right,
My anbcessed worknow, on me and greet.
  Thou houlss the world of more,
And all seant chnwhy being should bade,
When I would that zence as it efail)
Than thee that thou art confort's now, berind,
Eves thy sleed bitterue' thine every deat:
Thou sick your defice hass as firraters,
I sacked her blood a bespeyenows, from for 